In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from pathlib import Path
import os

from dotenv import load_dotenv

In [2]:
load_dotenv()
env_path = ('../.env')
load_dotenv(dotenv_path=env_path)

SPOTIFY_CLIENT = os.getenv('SPOTIFY_CLIENT')
SPOTIFY_SECRET = os.getenv('SPOTIFY_SECRET')
USERNAME = os.getenv('SPOTIFY_USERNAME')
redirect_uri = 'http://localhost:8888/callback/'

os.environ['SPOTIPY_CLIENT_ID']= SPOTIFY_CLIENT
os.environ['SPOTIPY_CLIENT_SECRET']= USERNAME
os.environ['SPOTIPY_REDIRECT_URI']= redirect_uri

print(SPOTIFY_CLIENT, SPOTIFY_SECRET, USERNAME)

a3ca004450a041b4b3e13c1f3bcc5ec5 b92d8f78722d44868c8b13dc4501ecd0 madathilvisakh


In [19]:
#authorizations
scope = 'user-library-read user-top-read playlist-modify-public playlist-read-private'


credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT, client_secret=SPOTIFY_SECRET) 
sp = spotipy.Spotify(client_credentials_manager=credentials_manager)
token = util.prompt_for_user_token(USERNAME, scope, SPOTIFY_CLIENT, SPOTIFY_SECRET, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", USERNAME) 

In [4]:
results = sp.current_user_saved_tracks()
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

for track in tracks:
    print(track['track']['name'])

Entrégala
Pretty Please
LA CANCIÓN
Professional
Adaptation
Kiss Land
Heat Waves
MONTERO (Call Me By Your Name)
Save Your Tears (with Ariana Grande) (Remix)
Kiss Me More (feat. SZA)
FEEL.
I Don't Dance
my ex's best friend (with blackbear)
Old Yellow Bricks
If You Were There, Beware
505
Right Side of My Neck
the WORLD
Kingston
Wait a Minute!
Way Out (feat. Big Sean)
21C/Delta
Raingurl
Love Story (Taylor’s Version)
Fkn Around (feat. Megan Thee Stallion)
Love It If We Made It
Don't Touch My Hair (feat. Sampha)
i finally understand
Goodie Bag
Leave The Door Open
We're Good
I Don't Care - Single Version
The Phoenix
My Songs Know What You Did In The Dark (Light Em Up)
The Only Difference Between Martyrdom and Suicide Is Press Coverage
Misery Business
Mauja Hi Mauja
Boomin
Last Time (feat. Snoh Aalegra)
Tú
Diamond Rings
Leader Of The Delinquents
Levitating
Morrow
On My Shit (feat. Joey Bada$$)
10%
Easy (with Kacey Musgraves feat. Mark Ronson)
Rainbow
World We Created
Heartbreak Anniversary
Bes

In [13]:
#creating a database of all songs and features
import pandas as pd

#songs = tracks["items"]

track_ids = []
track_names = []

for i in range(0, len(tracks)):
    #Removes the local tracks in your playlist if there is any
    if tracks[i]['track']['id'] != None: 
        track_ids.append(tracks[i]['track']['id'])
        track_names.append(tracks[i]['track']['name'])

features = []
for i in range(0,len(track_ids)):
    audio_features = sp.audio_features(track_ids[i])
    for track in audio_features:
        features.append(track)
        
playlist_df = pd.DataFrame(features, index = track_names)

playlist_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Entrégala,0.919,0.672,10,-3.404,0,0.1310,0.0637,0.014400,0.0597,0.425,117.002,audio_features,46lZpwebFWgqad5CX1iv6r,spotify:track:46lZpwebFWgqad5CX1iv6r,https://api.spotify.com/v1/tracks/46lZpwebFWgq...,https://api.spotify.com/v1/audio-analysis/46lZ...,158040,4
Pretty Please,0.906,0.474,1,-6.124,1,0.1940,0.0311,0.000005,0.2860,0.656,106.976,audio_features,5Pc594FhDA2Fa2prE75GT0,spotify:track:5Pc594FhDA2Fa2prE75GT0,https://api.spotify.com/v1/tracks/5Pc594FhDA2F...,https://api.spotify.com/v1/audio-analysis/5Pc5...,194607,4
LA CANCIÓN,0.754,0.646,7,-5.795,1,0.3170,0.1520,0.000018,0.1080,0.429,176.089,audio_features,0fea68AdmYNygeTGI4RC18,spotify:track:0fea68AdmYNygeTGI4RC18,https://api.spotify.com/v1/tracks/0fea68AdmYNy...,https://api.spotify.com/v1/audio-analysis/0fea...,242573,4
Professional,0.405,0.619,11,-8.920,0,0.0615,0.1620,0.000334,0.0788,0.234,120.092,audio_features,5ZicFGBDAi9J2YCVesboUp,spotify:track:5ZicFGBDAi9J2YCVesboUp,https://api.spotify.com/v1/tracks/5ZicFGBDAi9J...,https://api.spotify.com/v1/audio-analysis/5Zic...,368720,4
Adaptation,0.642,0.790,11,-6.800,0,0.0681,0.1510,0.000583,0.3050,0.613,114.960,audio_features,4pHQ9RYFhn3W0ha5KEuMnj,spotify:track:4pHQ9RYFhn3W0ha5KEuMnj,https://api.spotify.com/v1/tracks/4pHQ9RYFhn3W...,https://api.spotify.com/v1/audio-analysis/4pHQ...,283933,4


In [14]:
#keeping only relevant features

tracks_df = playlist_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]

tracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Entrégala,46lZpwebFWgqad5CX1iv6r,0.0637,0.919,158040,0.672,0.014400,10,0.0597,-3.404,0,0.1310,117.002,0.425
Pretty Please,5Pc594FhDA2Fa2prE75GT0,0.0311,0.906,194607,0.474,0.000005,1,0.2860,-6.124,1,0.1940,106.976,0.656
LA CANCIÓN,0fea68AdmYNygeTGI4RC18,0.1520,0.754,242573,0.646,0.000018,7,0.1080,-5.795,1,0.3170,176.089,0.429
Professional,5ZicFGBDAi9J2YCVesboUp,0.1620,0.405,368720,0.619,0.000334,11,0.0788,-8.920,0,0.0615,120.092,0.234
Adaptation,4pHQ9RYFhn3W0ha5KEuMnj,0.1510,0.642,283933,0.790,0.000583,11,0.3050,-6.800,0,0.0681,114.960,0.613


In [23]:
import json

#get short term df
results_short = sp.current_user_top_tracks(limit = 50, offset=0, time_range='short_term')['items']

for song in range(50):
        list = []
        list.append(results_short)
        with open('top50_data.json', 'w', encoding='utf-8') as f:
            json.dump(list, f, ensure_ascii=False, indent=4)

In [26]:
short_id = []
short_names = []

results_short = sp.current_user_top_tracks(limit = 50, offset=0, time_range='short_term')['items']

for i in range(0, len(results_short)):
    #Removes the local tracks in your playlist if there is any
    short_id.append(results_short[i]['uri'])
    short_names.append(results_short[i]['name'])

short_df = pd.DataFrame(short_id, index = short_names, columns = ['uri'])

short_df.head()

,uri
to the beginning,spotify:track:7Fn39khTcovMslPdNxx7b8
スパイラル,spotify:track:1o9Tn1pO1bVIJBEAcmG0If
三原色,spotify:track:4rfcYp6k3NG7eYUF8wG40U
DYING WISH,spotify:track:0nm0bdksXzUJwJaB4pbCa9
my ex's best friend (with blackbear),spotify:track:7kDUspsoYfLkWnZR7qwHZl


In [27]:
#get medium term df

results_medium = sp.current_user_top_tracks(limit=50,offset=0,time_range='medium_term')['items']

med_id = []
med_names = []

for i in range(0, len(results_medium)):
    #Removes the local tracks in your playlist if there is any
    med_id.append(results_medium[i]['uri'])
    med_names.append(results_medium[i]['name'])

med_df = pd.DataFrame(med_id, index = med_names, columns = ['uri'])

med_df.head()

,uri
Shoukei to Shikabane no Michi,spotify:track:7lcTpZtdTyfiIgVIpmTVut
Tequila Shots,spotify:track:30KctD1WsHKTIYczXjip5a
revenger,spotify:track:6olFvRvU0XqcxR7iss86Hv
Sad People,spotify:track:4nuAslShoN77tq12fzwjUq
RESISTER,spotify:track:2lO9sYbtyixyMIlj3TDcuU


In [28]:
#long term df

results_long = sp.current_user_top_tracks(limit=50,offset=0,time_range='long_term')['items']

long_id = []
long_names = []

for i in range(0, len(results_long)):
    #Removes the local tracks in your playlist if there is any
    long_id.append(results_long[i]['uri'])
    long_names.append(results_long[i]['name'])

long_df = pd.DataFrame(long_id, index = long_names, columns = ['uri'])

long_df.head()

,uri
Ordinary Life,spotify:track:15k2nBQJ0teDmPZHrOXL2N
On My Own,spotify:track:4D1eVq5AUILwjg3tAe7o6M
Reborn,spotify:track:4RVbK6cV0VqWdpCDcx3hiT
Faith,spotify:track:3orEOtd8CPL8GFlpRpKuoE
Shoukei to Shikabane no Michi,spotify:track:7lcTpZtdTyfiIgVIpmTVut


In [31]:
short_df['short_pts'] = 3
med_df['med_pts'] = 2
long_df['long_pts'] = 1

,uri,long_pts
Ordinary Life,spotify:track:15k2nBQJ0teDmPZHrOXL2N,1
On My Own,spotify:track:4D1eVq5AUILwjg3tAe7o6M,1
Reborn,spotify:track:4RVbK6cV0VqWdpCDcx3hiT,1
Faith,spotify:track:3orEOtd8CPL8GFlpRpKuoE,1
Shoukei to Shikabane no Michi,spotify:track:7lcTpZtdTyfiIgVIpmTVut,1
